In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from dotenv import load_dotenv
from utils import load_img_path_from_dir, img_to_array
from preprocess import balance_datasets
from model2 import init as model2_init, compile as model2_compile

In [4]:
balance_datasets()

🖼 Balance datasets
./Datasets/Dataset_Test/0
./Datasets/Dataset_Test/1
6
12
7
23
14
23
27
7
26
25
25
34
0
30
38
22
43
9
46
41
34
14
15
18
40
17
48
31
28
2
21
29
14
8
26
51
55
0
44
26
20
59
55
13
13
71
60
44
29
73
71
14
12
13
28
50
27
58
69
84
27
48
24
77
90
21
0
28
54
31
53
./Datasets/Dataset_Train/0
./Datasets/Dataset_Train/1
14
28
25
12
31
8
14
29
27
36
33
1
22
18
36
18
42
16
34
15
43
40
45
9
31
9
17
35
17
28
28
4
2
47
57
43
30
46
1
10
37
50
29
55
13
65
8
8
48
24
3
24
63
7
36
35
20
34
66
59
78
57
34
61
3
60
15
2
94
74
16
65
./Datasets/Dataset_Val/0
./Datasets/Dataset_Val/1
26
24
2
12
5
13
30
25
18
10
2
32
29
23
23
33
29
30
41
31
12
15
16
45
13
0
13
38
44
42
3
55
22
32
6
44
39
2
0
22
1
41
40
37
1
13
36
24
48
55
12
2
66
45
0
74
56
49
48
7
49
56
19
38
77
5
24
17
33
37
11


In [6]:
model = model2_init()
model = model2_compile(model)

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 255, 255, 32)      2432      
                                                                 
 conv2d_1 (Conv2D)           (None, 255, 255, 64)      51264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 63, 63, 128)       131200    
                                                                 
 conv2d_3 (Conv2D)           (None, 63, 63, 160)       327840    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 160)      0         
 2D)                                                    

In [6]:
dir_train = os.environ.get('TRAIN_PATH')
dir_test = os.environ.get('TEST_PATH')
dir_val = os.environ.get('VAL_PATH')

In [7]:
dataset_train = image_dataset_from_directory(
  dir_train,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255,255),
  batch_size=8)

Found 130 files belonging to 2 classes.


In [8]:
dataset_val = image_dataset_from_directory(
    dir_val,
    labels = "inferred",
    label_mode = "categorical",
    seed=123,
    image_size=(255,255),
    batch_size=8
)

Found 318 files belonging to 2 classes.


In [9]:
dataset_test = image_dataset_from_directory(
    dir_test,
    labels = 'inferred',
    label_mode = 'categorical',
    seed=123,
    image_size=(255,255),
    batch_size=8,
)

Found 130 files belonging to 2 classes.


In [10]:
# dataset_train = datagen.flow(dataset_train)

# Apply data augmentation and prefetch the data
# dataset_train = dataset_train.map(lambda x, y: (datagen.random_transform(x), y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [11]:
#es_recall = EarlyStoppingByRecall(monitor='val_recall', value=0.90, verbose=1)
es = EarlyStopping(patience=1,restore_best_weights=True)  # Early Stopping normal para la pérdida
model = model2_init()
model = model2_compile(model)
history = model.fit(dataset_train,validation_data=dataset_val,epochs=20,callbacks=[es])

Epoch 1/20


2023-12-16 19:47:07.050212: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


17/17 [==============================] - 20s 1s/step - loss: 88.1809 - recall_1: 0.5846 - val_loss: 0.5100 - val_recall_1: 0.9057
Epoch 2/20
17/17 [==============================] - 19s 1s/step - loss: 0.6705 - recall_1: 0.6846 - val_loss: 0.5923 - val_recall_1: 0.9057


In [13]:
# load the images from tests/0 and predict
img_tests_0 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/0')
predictions_O = []
for img in img_tests_0:
    img = img_to_array(img)
    predictions_O.append(model.predict(img))

1/1 [==============================] - 0s 37ms/step


In [16]:
# load the images from tests/1 and predict
img_tests_1 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/1')
predictions_1 = []
for img in img_tests_1:
    img = img_to_array(img)
    predictions_1.append(model.predict(img))

1/1 [==============================] - 0s 37ms/step


In [ ]:
# Obtener la categoria con más probabilidad
for res in predictions_O:
    dog_id = np.argmax(res.max())
    print(dog_id)

In [ ]:
for res in predictions_1:
    dog_id = np.argmax(res.max())
    print(dog_id)

In [ ]:
len(predictions_1)

In [ ]:
len(predictions_O)